In [1]:
!git clone https://github.com/openai/shap-e.git


Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 15.90 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
%cd shap-e

/content/shap-e


In [3]:
!pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-97och_my/clip_e40495a8c3754a82bf7b29145be1a17b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-97och_my/clip_e40495a8c3754a82bf7b29145be1a17b
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
 

In [9]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

In [27]:
# Define 4 prompts to generate 4 examples
prompts = [
    "a robot",
    "a futuristic car",
    "a medieval sword",
    "a flying drone"
]
batch_size = len(prompts)
guidance_scale = 15.0

# Generate latents for each prompt
latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=prompts),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)




  0%|          | 0/64 [00:00<?, ?it/s]

In [29]:
render_mode = 'nerf' # or 'stf'
size = 64

cameras = create_pan_cameras(size, device)

for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))  # Shows a rotating GIF of the renders (in a notebook)
    print(f"Displayed 3D renders for prompt: {prompts[i]}")

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAK2urq2ura2tra2trK2srKyurqytrKysraysrKqsrKmsrKyrrK…

Displayed 3D renders for prompt: a robot


HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAAC34AC33wC24AC13wC13gCt1gCs1wCs1gCr2QCr1wCp1gCk0w…

Displayed 3D renders for prompt: a futuristic car


HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAALWyYrOuZa2qXayqX6upYqinYKimXaemXaalXaChbJ+ha6GgYZ…

Displayed 3D renders for prompt: a medieval sword


HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAACh+ACR4QCP4gCN5gCM5gCN4QCL5QCL4ACK3gCL0QCI3QCI1g…

Displayed 3D renders for prompt: a flying drone


In [34]:
!pip install trimesh


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.0/709.0 kB 43.7 MB/s eta 0:00:00


In [35]:
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.obj', 'w') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_obj(f)
